In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import matplotlib.pyplot as plt
import netCDF4
import xarray as xr
import metpy
from datetime import datetime as dt, timedelta
#import datetime as dt
from metpy.units import units
import scipy.ndimage as ndimage
from metpy.plots import USCOUNTIES
import cartopy
from scipy.ndimage.filters import generic_filter as gf
from matplotlib.animation import FuncAnimation
from IPython import display
import matplotlib
import os
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator, LongitudeLocator)
import matplotlib.colors as colors
from gradient import Gradient
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import metpy.calc as mpcalc
import time as t
import re
import fnmatch
from bs4 import BeautifulSoup
import wget
import cfgrib
import matplotlib as mpl
import cmap_atmos
import cmap_radar
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
from matplotlib.ticker import StrMethodFormatter
import glob, os

import requests

#%matplotlib inline

# Create an animation
from matplotlib.animation import ArtistAnimation

ERROR 1: PROJ: proj_create_from_database: Open of /knight/anaconda_jan21/envs/oct21/share/proj failed
/tmp/ipykernel_16181/3213611649.py:14: DeprecationWarning: Please use `generic_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import generic_filter as gf


In [2]:
plot_regions_dict = {
    'neus': {'central_longitude': -75.75, 'extent': [-85.5,-66,36,49], 'plot_counties': True, 'county_lw': 0.5, 'county_scale': '20m', 'windbarb_skip': 3},
    'seus': {'central_longitude': -86, 'extent': [-98,-74,22,38], 'plot_counties': True, 'county_lw': 0.5, 'county_scale': '20m', 'windbarb_skip': 3},
    'ncus': {'central_longitude': -93.25, 'extent': [-104.5,-82,35,50], 'plot_counties': True, 'county_lw': 0.5, 'county_scale': '20m', 'windbarb_skip': 3},
    'scus': {'central_longitude': -97.75, 'extent': [-110.5,-85,25,42], 'plot_counties': True, 'county_lw': 0.5, 'county_scale': '20m', 'windbarb_skip': 3},
    'nwus': {'central_longitude': -116.5, 'extent': [-130,-103,36,54], 'plot_counties': True, 'county_lw': 0.5, 'county_scale': '20m', 'windbarb_skip': 3},
    'swus': {'central_longitude': -116.5, 'extent': [-130,-103,27,45], 'plot_counties': True, 'county_lw': 0.5, 'county_scale': '20m', 'windbarb_skip': 3},
    'east': {'central_longitude': -82.25, 'extent': [-102.5,-62,23,50], 'plot_counties': True, 'county_lw': 0.25, 'county_scale': '20m', 'windbarb_skip': 5},
    'cent': {'central_longitude': -100, 'extent': [-119.5,-82,25,50], 'plot_counties': True, 'county_lw': 0.25, 'county_scale': '20m', 'windbarb_skip': 5},
    'west': {'central_longitude': -115, 'extent': [-132.5,-98,27,50], 'plot_counties': True, 'county_lw': 0.25, 'county_scale': '20m', 'windbarb_skip': 5},
    'ak': {'central_longitude': -115, 'extent': [-170,-130,50,75], 'plot_counties': True, 'county_lw': 0.25, 'county_scale': '20m', 'windbarb_skip': 5},
    'usa': {'central_longitude': -112.5, 'extent': [-131,-63,20.2,55], 'plot_counties': False, 'windbarb_skip': 7}
    }

In [3]:
def get_projection(plot_region, plot_regions_dict = plot_regions_dict, projection = 'PlateCarree'):
    """Get projection for plot centered on an input region (plot_region), defaults to LambertConformal"""
    if plot_region not in plot_regions_dict.keys():
        raise Exception(f'"{plot_region}" region not defined (valid regions: {list(plot_regions_dict.keys())})')
    if projection == 'LambertConformal':
        return ccrs.LambertConformal(central_longitude = plot_regions_dict[plot_region]['central_longitude'])
    elif projection == 'PlateCarree':
        return ccrs.PlateCarree()
    elif projection == 'Mercator':
        return ccrs.Mercator()
    else:
        raise Exception(f'invalid projection "{projection}" -- add to function')
        
#if plot_region == 'usa':
#    projection == ccrs.Mercator()
        
def plot_geog_single(ax, plot_region, plot_regions_dict = plot_regions_dict, 
                     geog_color = '0.2', geog_lw = 1.2, geog_scale = '10m', plot_lakes = 'fill', county_color = 'darkgrey',
                     wrf_variable = None):
    """Helper function for plotting cartopy geography on an axis
    most things are pretty self-explanatory to set major plot attributes,
    wrf_variable can be any variable extracted from wrfout file using wrf-python getvar() so plot border extent can be set for CONUS-scale plots"""
    #Download and plot states and coastlines
    #geog_scale options: 10m, 50m, 110 m (fine, medium, coarse)

    #select predefined plot options for given region
    if plot_region not in plot_regions_dict.keys():
        raise Exception(f'"{plot_region}" region not defined (valid regions: {list(plot_regions_dict.keys())})')
    plot_opts = plot_regions_dict[plot_region]
    if plot_opts['plot_counties'] == True:
        ax.add_feature(USCOUNTIES.with_scale(plot_opts['county_scale']),edgecolor=county_color,zorder=3.0, linewidth = plot_opts['county_lw']) #Add US counties
    ax.add_feature(cfeature.OCEAN.with_scale(geog_scale),edgecolor='face',facecolor=cfeature.COLORS['water'])
    ax.add_feature(cfeature.LAND.with_scale(geog_scale),edgecolor='face',facecolor=cfeature.COLORS['land'])
    ax.add_feature(cfeature.BORDERS.with_scale(geog_scale),linewidth=geog_lw,edgecolor=geog_color,zorder=3.1,linestyle='--')
    ax.add_feature(cfeature.COASTLINE.with_scale(geog_scale),linewidth=geog_lw,edgecolor=geog_color,facecolor='none',zorder=3)
    ax.add_feature(cfeature.STATES.with_scale(geog_scale),linewidth=geog_lw-0.5,edgecolor=geog_color,facecolor='none',zorder=3.1)
    if plot_lakes in ['filled', 'both']:
        ax.add_feature(cfeature.LAKES.with_scale(geog_scale),linewidth=geog_lw-0.5,edgecolor='face',facecolor=cfeature.COLORS['water'],) #zorder=2.9) #Filled lakes, no outlines 
    if plot_lakes in ['outline', 'both']:
        ax.add_feature(cfeature.LAKES.with_scale(geog_scale),linewidth=geog_lw-0.5,edgecolor=geog_color,facecolor='none',zorder=3) #Lake outlines only
    #ax.add_feature(cfeature.RIVERS.with_scale(geog_scale),linewidth=2.0) #Filled rivers, no outlines 

    #Set extent to focus on smaller region of domain
    if plot_region != 'conus':
        ax.set_extent(plot_opts['extent'], ccrs.PlateCarree()) #lonmin, lonmax, latmin, latmax of plot extent
    else:
        if wrf_variable is None:
            raise Exception('specify any variable from wrfout file to get native full-domain plot extent')
        #Set map bounds to full extent of model domain
        ax.set_xlim(cartopy_xlim(wrf_variable))
        ax.set_ylim(cartopy_ylim(wrf_variable))


In [4]:
def plot_maxmin_points(lon, lat, data, extrema, nsize, symbol, color='k',
                       plotValue=True, transform=None):
    """
    This function will find and plot relative maximum and minimum for a 2D grid. The function
    can be used to plot an H for maximum values (e.g., High pressure) and an L for minimum
    values (e.g., low pressue). It is best to used filetered data to obtain  a synoptic scale
    max/min value. The symbol text can be set to a string value and optionally the color of the
    symbol and any plotted value can be set with the parameter color
    lon = plotting longitude values (2D)
    lat = plotting latitude values (2D)
    data = 2D data that you wish to plot the max/min symbol placement
    extrema = Either a value of max for Maximum Values or min for Minimum Values
    nsize = Size of the grid box to filter the max and min values to plot a reasonable number
    symbol = String to be placed at location of max/min value
    color = String matplotlib colorname to plot the symbol (and numerica value, if plotted)
    plot_value = Boolean (True/False) of whether to plot the numeric value of max/min point
    The max/min symbol will be plotted on the current axes within the bounding frame
    (e.g., clip_on=True)
    """
    from scipy.ndimage.filters import maximum_filter, minimum_filter

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        ax1.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], symbol, color=color, size=24,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        ax1.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]],
                '\n' + str(np.int(data[mxy[i], mxx[i]])),
                color=color, size=12, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)

### Defining the run to use

In [11]:
bounds_temp = np.arange(-60,131,1)
norm_temp = colors.BoundaryNorm(boundaries=bounds_temp, ncolors=191)
obj_temp = Gradient([['#36365e',-60.0],['#5097d9',-50.0]],
               [['#5097d9',-50.0],['#2f8764',-40.0]],
               [['#2f8764',-40.0],['#33f5f5',-30.0]],
               [['#33f5f5',-30.0],['#a422b3',-20.0]],
               [['#a422b3',-20.0],['#47064f',-10.0]],
               [['#47064f',-10.0],['#efedf0',0.0]],
               [['#7e7cf7',0.0],['#020b96',10.0]],
               [['#020b96',10.0],['#0047ab',20.0]],
               [['#0047ab',20.0],['#00fffb',32.0]],
               [['#15d15a',32.0],['#00751b',40.0]],
               [['#00751b',40.0],['#67cf00',50.0]],
               [['#67cf00',50.0],['#fbff00',60.0]],
               [['#fbff00',60.0],['#ffae00',70.0]],
               [['#ffae00',70.0],['#de0000',80.0]],
               [['#de0000',80.0],['#96002d',90.0]],
               [['#96002d',90.0],['#e096ad',100.0]],
               [['#e096ad',100.0],['#ad727a',110.0]],
               [['#ad727a',110.0],['#78344b',120]],
               [['#78344b',120.0],['#521d2c',130]])
cmap_temp = obj_temp.get_cmap(bounds_temp) #Retrieve colormap
colors_temp = obj_temp.colors #Retrieve list of hex color values

bounds_gust = np.arange(10,150,1)
norm_gust = colors.BoundaryNorm(boundaries=bounds_gust, ncolors=len(bounds_gust))
obj_gust = Gradient(
               [['#bfbfbf',10.0],['#828282',20.0]],
               [['#026d9c',20.0],['#43c2fa',39.0]],
               [['#016309',39.0],['#81fc8b',50.0]],
               [['#fcd381',50.0],['#825701',74.0]],
               [['#800501',74.0],['#fa0a02',95.0]],
               [['#592601',95.0],['#b87442',110.0]],
               [['#6b004b',110.0],['#fc03b2',129.0]],
               [['#010575',129.0],['#343bfa',156.0]],)
cmap_gust = obj_gust.get_cmap(bounds_gust) #Retrieve colormap

bounds_prate = [0.005,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.12,0.14,0.16,0.18,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.6,0.7,0.8,0.9,1]
norm_prate = colors.BoundaryNorm(boundaries=bounds_prate, ncolors=len(bounds_prate))
obj_prate = Gradient(
               [['#78de93',0.005],['#014012',0.2]],
               [['#f5e902',0.2],['#f02805',0.5]],
                [['#f02805',0.5],['#eb02a5',1]])
cmap_prate = obj_prate.get_cmap(bounds_prate) #Retrieve colormap
bounds_w = [0.005,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.12,0.14,0.16,0.18,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55]
norm_w = colors.BoundaryNorm(boundaries=bounds_w, ncolors=len(bounds_w))
obj_snow = Gradient(
               [['#ffffff',0.005],['#7c78fa',0.1]],
               [['#7c78fa',0.1],['#0702a3',0.2]],
                [['#0702a3',0.2],['#9000c9',0.4]],
                [['#9000c9',0.4],['#ff03fb',0.55]])
cmap_snow_c = obj_snow.get_cmap(bounds_w) #Retrieve colormap

obj_frzr = Gradient(
               [['#fcacfb',0.005],['#f000d4',0.2]],
[['#f000d4',0.2],['#69015c',0.55]])
cmap_frzr_c = obj_frzr.get_cmap(bounds_w) #Retrieve colormap

obj_icep = Gradient(
               [['#d299f7',0.005],['#7200d6',0.2]],
[['#7200d6',0.2],['#380169',0.55]])
cmap_icep_c = obj_icep.get_cmap(bounds_w) #Retrieve colormap

#ptype_colors = ['#00FFFFFF','tab:blue','darkslateblue','deeppink','tab:green']
ptype_colors_fzra = [(1,1,1), 'deeppink']
cmap_ptype_ext_fzra = mpl.colors.ListedColormap(ptype_colors_fzra)
my_cmap = cmap_ptype_ext_fzra(np.arange(cmap_ptype_ext_fzra.N))
my_cmap[:,-1] = np.linspace(0, 1, cmap_ptype_ext_fzra.N)
cmap_fzra = ListedColormap(my_cmap)
#norm_ptype_ext_fzra = mpl.colors.BoundaryNorm([-1,0,1],ncolors = len(ptype_colors_fzra))

ptype_colors_snow = [(1,1,1), 'tab:blue']
cmap_ptype_ext_snow = mpl.colors.ListedColormap(ptype_colors_snow)
my_cmap = cmap_ptype_ext_snow(np.arange(cmap_ptype_ext_snow.N))
my_cmap[:,-1] = np.linspace(0, 1, cmap_ptype_ext_snow.N)
cmap_snow = ListedColormap(my_cmap)

ptype_colors_rain = [(1,1,1), 'tab:green']
cmap_ptype_ext_rain = mpl.colors.ListedColormap(ptype_colors_rain)
my_cmap = cmap_ptype_ext_rain(np.arange(cmap_ptype_ext_rain.N))
my_cmap[:,-1] = np.linspace(0, 1, cmap_ptype_ext_rain.N)
cmap_rain = ListedColormap(my_cmap)

ptype_colors_icep = [(1,1,1), 'mediumslateblue']
cmap_ptype_ext_icep = mpl.colors.ListedColormap(ptype_colors_icep)
my_cmap = cmap_ptype_ext_icep(np.arange(cmap_ptype_ext_icep.N))
my_cmap[:,-1] = np.linspace(0, 1, cmap_ptype_ext_icep.N)
cmap_icep = ListedColormap(my_cmap)

ptype_colors_zr_ip = [(1,1,1), 'darkmagenta']
cmap_ptype_ext_zrip = mpl.colors.ListedColormap(ptype_colors_zr_ip)
my_cmap = cmap_ptype_ext_zrip(np.arange(cmap_ptype_ext_zrip.N))
my_cmap[:,-1] = np.linspace(0, 1, cmap_ptype_ext_zrip.N)
cmap_zr_ip = ListedColormap(my_cmap)

ptype_colors_sn_ip = [(1,1,1), 'darkslateblue']
cmap_ptype_ext_snip = mpl.colors.ListedColormap(ptype_colors_sn_ip)
my_cmap = cmap_ptype_ext_snip(np.arange(cmap_ptype_ext_snip.N))
my_cmap[:,-1] = np.linspace(0, 1, cmap_ptype_ext_snip.N)
cmap_sn_ip = ListedColormap(my_cmap)

ptype_colors_sn_ra = [(1,1,1), 'darkturquoise']
cmap_ptype_ext_snra = mpl.colors.ListedColormap(ptype_colors_sn_ra)
my_cmap = cmap_ptype_ext_snra(np.arange(cmap_ptype_ext_snra.N))
my_cmap[:,-1] = np.linspace(0, 1, cmap_ptype_ext_snra.N)
cmap_sn_ra = ListedColormap(my_cmap)

ptype_colors_ra_ip = [(1,1,1), 'mediumaquamarine']
cmap_ptype_ext_raip = mpl.colors.ListedColormap(ptype_colors_ra_ip)
my_cmap = cmap_ptype_ext_raip(np.arange(cmap_ptype_ext_raip.N))
my_cmap[:,-1] = np.linspace(0, 1, cmap_ptype_ext_raip.N)
cmap_ip_ra = ListedColormap(my_cmap)

ptype_colors_zr_sn = [(1,1,1), 'mediumorchid']
cmap_ptype_ext_zrsn = mpl.colors.ListedColormap(ptype_colors_zr_sn)
my_cmap = cmap_ptype_ext_zrsn(np.arange(cmap_ptype_ext_zrsn.N))
my_cmap[:,-1] = np.linspace(0, 1, cmap_ptype_ext_zrsn.N)
cmap_zr_sn = ListedColormap(my_cmap)

ptype_colors_zr_ip_sn = [(1,1,1), 'violet']
cmap_ptype_ext_zripsn = mpl.colors.ListedColormap(ptype_colors_zr_ip_sn)
my_cmap = cmap_ptype_ext_zripsn(np.arange(cmap_ptype_ext_zripsn.N))
my_cmap[:,-1] = np.linspace(0, 1, cmap_ptype_ext_zripsn.N)
cmap_zr_ip_sn = ListedColormap(my_cmap)

ptype_colors_ra_ip_sn = [(1,1,1), 'cyan']
cmap_ptype_ext_raipsn = mpl.colors.ListedColormap(ptype_colors_ra_ip_sn)
my_cmap = cmap_ptype_ext_raipsn(np.arange(cmap_ptype_ext_raipsn.N))
my_cmap[:,-1] = np.linspace(0, 1, cmap_ptype_ext_raipsn.N)
cmap_ra_ip_sn = ListedColormap(my_cmap)

#Greating the colorbar for the ASOS observations

ptype_colors = [(1,1,1,1),'tab:blue','mediumslateblue','darkslateblue','deeppink','mediumorchid','darkmagenta','violet','tab:green',
                'darkturquoise','mediumaquamarine','cyan','grey']
cmap_ptype_ext = mpl.colors.ListedColormap(ptype_colors)
norm_ptype_ext = mpl.colors.BoundaryNorm([-1,0,1,2,3,5,6,8,9,10,11,12,13,14],ncolors = len(ptype_colors))
ptype_ticks = [-0.5,0.5,1.5,2.5,4,5.5,7,8.5,9.5,10.5,11.5,12.5,13.5]
ptype_labels = ['NP','SN','IP','IP/SN','ZR','ZR/SN','ZR/IP','ZR/IP/SN','RA','RA/SN','RA/IP','RA/IP/SN','UP']

cbar_ptype = mpl.cm.ScalarMappable(norm = norm_ptype_ext, cmap = cmap_ptype_ext)
cbar_ptype.set_array([])

#var_name = ['h5_vort', 'h7_vort','h85_vort','h850_temp','h700_temp','2m_temp', '2m_dpt', 'mslp', 'acp', 'cat_precip', '250_wind', 'precip_rate','ptype_rate', 'gust']
var_name = ['precip_rate','ptype_rate','max_gust']



plot_region = 'usa'

url = 'https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')

urls_gfs = []
for link in soup.find_all('a'):
    #print(link.get('href'))
    urls_gfs.append(link.get('href'))
    
date = urls_gfs[0][-8::]

url = url+f'?dir=%2Fgfs.{date}'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
urls_gefs = []
for link in soup.find_all('a'):
    #print(link.get('href'))
    urls_gefs.append(link.get('href'))
    
time = urls_gefs[0]
time = time[-2::]
time

#start = t.time()
for x in range(96,97,6):
    ################# GRABBING THE DATA AND THE RAW GRIB FILES FROM THE NOMADS NCEP SERVER################
    if len(str(x)) == 1:
        x2 = f'00{x}'
    if len(str(x)) == 2:
        x2 = f'0{x}'
    if len(str(x)) == 3:
        x2 = x
    
    #assigning lat lon to plot
    lats = np.arange(20,55.5,0.25)
    lons = np.arange(229,297.5,0.25)

    
    ds = wget.download(f'https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?file=gfs.t{time}z.pgrb2.0p25.f{x2}&lev_2_m_above_ground=on&lev_10_m_above_ground=on&lev_250_mb=on&lev_500_mb=on&lev_700_mb=on&lev_850_mb=on&lev_1000_mb=on&lev_mean_sea_level=on&lev_surface=on&var_ACPCP=on&var_APCP=on&var_CFRZR=on&var_CICEP=on&var_CRAIN=on&var_CSNOW=on&var_DPT=on&var_HGT=on&var_ABSV=on&var_MSLET=on&var_PRATE=on&var_TMP=on&var_UGRD=on&var_VGRD=on&subregion=&leftlon=229&rightlon=297.5&toplat=55.5&bottomlat=20&dir=%2Fgfs.{date}%2F{time}%2Fatmos')


    
    for var in var_name:
    

        ########## SET UP FIGURE ##################################################
        if plot_region == 'usa':
            fig, ax1 = plt.subplots(figsize = (12,9), dpi = 120, subplot_kw = {'projection': ccrs.Mercator()}, facecolor = 'white')
        else:
            fig, ax1 = plt.subplots(figsize = (12,9), dpi = 120, subplot_kw = {'projection': get_projection(plot_region)}, facecolor = 'white')
        plot_lakes = 'filled' if plot_region == 'conus' else 'both'    
        plot_geog_single(ax1, plot_region = plot_region, plot_lakes = plot_lakes)

 ########## PLOTTING #######################################################
        proj = ccrs.PlateCarree()

#################### VORTICITY #############################################
        if 'vort' in var:
            if 'h5' in var:
                lev=500.0
                levels = np.arange(480,620,6)
            if 'h85' in var:
                lev=850.0
                levels = np.arange(0,300,5)
            if 'h7' in var:
                lev=700.0
                levels = np.arange(0,500,4)
                
            ds_avort =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'isobaricInhPa'}).sel(isobaricInhPa = lev)
            
            valid_time = ds_avort['valid_time']
            init_time = ds_avort['time']
            #data1 = ds_temp.metpy.parse_cf()

            init_time = init_time.values
            init_time = dt.utcfromtimestamp(init_time.tolist()/1e9)
            init_time_str = init_time.strftime('%Y-%m-%d %H:%MZ')
            init_time_str_save = init_time.strftime('%Y-%m-%d-%H')
            valid_time = valid_time.values
            valid_time = dt.utcfromtimestamp(valid_time.tolist()/1e9)
            valid_time_str = valid_time.strftime('%Y-%m-%d-%H:%MZ')
            fc_hr = valid_time-init_time
            fc_hr = fc_hr/3600
            fc_hr = fc_hr.seconds
                            #creating directory to store the images
            imgdir = f'images/{var}/{plot_region}/{init_time_str_save}/'
            if not os.path.exists(imgdir):
                os.makedirs(imgdir)
            
            
            h = ds_avort['gh'].squeeze()*.1
            av = ds_avort['absv'].squeeze()*1e5
            u = ds_avort['u'].squeeze()*1.94384449
            v = ds_avort['v'].squeeze()*1.94384449
            
            #Wind Slice
            lon, lat = h.metpy.coordinates('x', 'y')
            
            #wind_slice = slice(5,-5,15)
            skip = int(0.8*plot_regions_dict[plot_region]['windbarb_skip'])
            wind_slice = slice(None, None, skip)
            
            
            #sigma = 0 # this depends on how noisy your data is, adjust as necessary
            #av = mpcalc.smooth_gaussian(av, sigma)

            h5c = ax1.contour(lon,lat,h,colors='black', levels = levels,linewidths=1.25, zorder=995, transform=proj)
            ax1.clabel(h5c, inline_spacing=0.2, fontsize='medium', zorder=999)

            a5c = ax1.contourf(lon,lat,av,cmap='inferno_r',levels=range(12,71,2),extend='max', zorder=2, transform=proj)

            lon = x.values
            lat = y.values
            u = u.values
            v = v.values

            ax1.barbs(lon[wind_slice],lat[wind_slice],u[wind_slice,wind_slice],v[wind_slice,wind_slice], length=5, linewidth=0.35, color='dimgrey', zorder=998, transform=proj)

            a5cb = fig.colorbar(a5c, orientation = 'horizontal', aspect = 80, ax = ax1, pad = 0.02,
                                extendrect=False, ticks = range(15,71,5), fraction=0.046)

            a5cb.set_label('500mb Absolute Vorticity ($10^{-5}s^{-1}$)', fontsize = 12, zorder=999)

            ax1.set_title('\n GFS Init: '+init_time_str,fontsize=8,loc='right')

            plt.title('GFS 500mb Geopotential Height (dam), Absolute Vorticity ($10^{-5}s^{-1}$), and Wind (kt)\n', fontsize=11, fontweight='bold',loc='left')
            plt.suptitle('Valid: '+valid_time_str+', Forecast Hour: [' +str(fc_hr)+ ']\n', x=0.232, y=0.883, fontsize=8, zorder=999)
            print('Done Plotting')
            
################## TEMPERATURES ##############################
        if 'temp' in var:

            
            #opening the data
            if '2m' in var:
                ds_temp =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'heightAboveGround'})
                ds_h =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'meanSea'})
                #u = ds_temp['u10'].squeeze()*1.94384449
                #v = ds_temp['v10'].squeeze()*1.94384449
                h = ds_h['mslet'].squeeze()*.01
                temp = (ds_temp['t2m'].squeeze()-273.15)*9/5+32
                title = '2-meter Air'
                title2 = 'MSLP'
                levels = np.arange(0,1080,4)
            if 'h85' in var:
                lev=850.0
                ds_temp = xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'isobaricInhPa'}).sel(isobaricInhPa = lev)
                ds_avort =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'isobaricInhPa'}).sel(isobaricInhPa = lev)
                temp = ds_temp['t']
                temp = (temp.squeeze()-273.15)*9/5+32
                u = ds_avort['u'].squeeze()*1.94384449
                v = ds_avort['v'].squeeze()*1.94384449
                h = ds_avort['gh'].squeeze()*.1
                title = '850mb'
                title2 = 'Heights'
                levels = np.arange(0,300,5)
            if 'h7' in var:
                lev=700.0
                ds_temp = xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'isobaricInhPa'}).sel(isobaricInhPa = lev)
                ds_avort =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'isobaricInhPa'}).sel(isobaricInhPa = lev)
                temp = ds_temp['t']
                temp = (temp.squeeze()-273.15)*9/5+32
                u = ds_avort['u'].squeeze()*1.94384449
                v = ds_avort['v'].squeeze()*1.94384449
                h = ds_avort['gh'].squeeze()*.1
                title = '700mb'
                title2 = 'Heights'
                levels = np.arange(0,500,4)
                
                
            valid_time = ds_temp['valid_time']
            init_time = ds_temp['time']
            #data1 = ds_temp.metpy.parse_cf()

            init_time = init_time.values
            init_time = dt.utcfromtimestamp(init_time.tolist()/1e9)
            init_time_str = init_time.strftime('%Y-%m-%d %H:%MZ')
            init_time_str_save = init_time.strftime('%Y-%m-%d-%H')
            valid_time = valid_time.values
            valid_time = dt.utcfromtimestamp(valid_time.tolist()/1e9)
            valid_time_str = valid_time.strftime('%Y-%m-%d-%H:%MZ')
            fc_hr = valid_time-init_time
            fc_hr = fc_hr/3600
            fc_hr = fc_hr.seconds
            imgdir = f'images/{var}/{plot_region}/{init_time_str_save}/'
            if not os.path.exists(imgdir):
                os.makedirs(imgdir)
            
            lon,lat = temp.metpy.coordinates('x', 'y')
            lat, lon = xr.broadcast(y, x)
            #For filled contour
            #CT = ax1.pcolormesh(lon,lat,t2m, cmap=cmap_temp, norm=norm_temp, shading='goaround', zorder=2, transform=proj)
            CT = ax1.contourf(x.values,y.values,temp, cmap=cmap_temp, norm=norm_temp, zorder=2, levels=bounds_temp, transform=proj)
            frzC = ax1.contour(lon,lat,temp, levels=[32], colors='#e83553', zorder=2, transform=proj)
            
            hc = ax1.contour(lon,lat,h,colors='black', levels = levels,linewidths=1.25, zorder=995, transform=proj)
            ax1.clabel(hc, inline_spacing=0.2, fontsize='medium', zorder=999)
            
             #Wind Slice
            lon,lat = h.metpy.coordinates('x', 'y')

            #skip = int(0.8*plot_regions_dict[plot_region]['windbarb_skip'])
            #wind_slice = slice(None, None, skip)
            
            #lon = x.values
            #lat = y.values
            #u = u.values
            #v = v.values

            #ax1.barbs(lon[wind_slice],lat[wind_slice],u[wind_slice,wind_slice],v[wind_slice,wind_slice], length=5, linewidth=0.35, color='dimgrey', zorder=998, transform=proj)

            #For unfilled contour
            #sigma = 0.0 # this depends on how noisy your data is, adjust as necessary
            #Tsmth = mpcalc.smooth_gaussian(temp, sigma)
            #CT = ax1.contour(lon,lat,Tsmth,levels=np.arange(-60,130+10,10), transform = ccrs.PlateCarree(), cmap=cmap, norm=norm)
            #ax1.clabel(CT, inline_spacing=0.2)

            #Can keep these lines uncommented out so we get a filled colorbar no matter if we use contour or contourf
            sm = plt.cm.ScalarMappable(norm=norm_temp, cmap = CT.cmap)
            sm.set_array([])

            cbar = fig.colorbar(sm, orientation = 'horizontal', aspect = 80, ax = ax1, pad = 0.02,
                    extendrect=False, ticks = range(-60,131,10), fraction=0.046, shrink=0.95)
            cbar.set_label('Temperature ($^\circ$F)', fontsize=12)

            ax1.set_title('\n GFS Init: '+init_time_str,fontsize=8,loc='right')

            plt.title(f'GFS {title} Temperature [deg F, fill], Wind Barbs [kt], {title2}\n', fontsize=11, fontweight='bold',loc='left')
            #plt.title('GFS 500mb Geopotential Height (dam), Absolute Vorticity ($10^{-5}s^{-1}$), and Wind (kt)\n', fontsize=11, fontweight='bold',loc='left')
            plt.suptitle('Valid: '+valid_time_str+', Forecast Hour: [' +str(fc_hr)+ ']\n', x=0.232, y=0.883, fontsize=8, zorder=999)
        
            print('Done Plotting')
            
################## MSLP ANOM ##############################
        if 'mslp' in var:

            ds_h =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'meanSea'})
            #ds_anom =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'meanSea','paramId':171151}).sel(latitude=lats, longitude=lons)
            #u = ds_u['u10'].squeeze()*1.94384449
            #v = ds_v['v10'].squeeze()*1.94384449
            #h = ds_h['mslet'].squeeze()*.01
            #anom = ds_anom['msla'].squeeze()*.01
            #temp = (ds_temp['t2m'].squeeze()-273.15)*9/5+32
            title = 'MSLP Anomaly'
            title2 = 'MSLP [contour]'
            levels = np.arange(0,1080,2)

                
                
            valid_time = ds_h['valid_time']
            init_time = ds_h['time']
            #data1 = ds_temp.metpy.parse_cf()

            init_time = init_time.values
            init_time = dt.utcfromtimestamp(init_time.tolist()/1e9)
            init_time_str = init_time.strftime('%Y-%m-%d %H:%MZ')
            init_time_str_save = init_time.strftime('%Y-%m-%d-%H')
            valid_time = valid_time.values
            valid_time = dt.utcfromtimestamp(valid_time.tolist()/1e9)
            valid_time_str = valid_time.strftime('%Y-%m-%d-%H:%MZ')
            fc_hr = valid_time-init_time
            fc_hr = fc_hr/3600
            fc_hr = fc_hr.seconds
            imgdir = f'images/{var}/{plot_region}/{init_time_str_save}/'
            if not os.path.exists(imgdir):
                os.makedirs(imgdir)
            
            lon,lat = h.metpy.coordinates('x', 'y')

            #For filled contour
            
            #norm = mpl.colors.Normalize(-30,30,1)
            #CT = ax1.pcolormesh(lon,lat,t2m, cmap=cmap_temp, norm=norm_temp, shading='goaround', zorder=2, transform=proj)
            #CT = ax1.contourf(lon,lat,anom, cmap='coolwarm', norm=norm, zorder=2, levels=np.arange(-30,30,1), transform=proj)

            
            hc = ax1.contour(lon,lat,h,colors='k', levels = levels,linewidths=1, zorder=995, transform=proj)
            ax1.clabel(hc, inline_spacing=0.2, fontsize='small', zorder=999)
            
            
            #plot_maxmin_points(lon,lat, h, 'max', 50, symbol='H', color='b',  transform=proj)
            #plot_maxmin_points(lon,lat, h, 'min', 25, symbol='L', color='r', transform=proj)
            
            #For unfilled contour
            #sigma = 0.0 # this depends on how noisy your data is, adjust as necessary
            #Tsmth = mpcalc.smooth_gaussian(temp, sigma)
            #CT = ax1.contour(lon,lat,Tsmth,levels=np.arange(-60,130+10,10), transform = ccrs.PlateCarree(), cmap=cmap, norm=norm)
            #ax1.clabel(CT, inline_spacing=0.2)

            #Can keep these lines uncommented out so we get a filled colorbar no matter if we use contour or contourf
            #sm = plt.cm.ScalarMappable(norm=norm, cmap = CT.cmap)
            #sm.set_array([])

            #cbar = fig.colorbar(sm, orientation = 'horizontal', aspect = 80, ax = ax1, pad = 0.02,
            #        extendrect=False, ticks = range(-60,131,10), fraction=0.046)
            #cbar.set_label('Temperature ($^\circ$F)', fontsize=12)

            ax1.set_title('\n GFS Init: '+init_time_str,fontsize=8,loc='right')

            plt.title(f'GFS {title} [hPa, fill], 10m Wind Barbs [kt], {title2}\n', fontsize=11, fontweight='bold',loc='left')
            #plt.title('GFS 500mb Geopotential Height (dam), Absolute Vorticity ($10^{-5}s^{-1}$), and Wind (kt)\n', fontsize=11, fontweight='bold',loc='left')
            plt.suptitle('Valid: '+valid_time_str+', Forecast Hour: [' +str(fc_hr)+ ']\n', x=0.232, y=0.883, fontsize=8, zorder=999)
        
            print('Done Plotting')

##################################### DEWPOINT ##################################
            
        if var == '2m_dpt':
            
            ds_dpt =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'heightAboveGround'})
            
            valid_time = ds_dpt['valid_time']
            init_time = ds_dpt['time']
            #data1 = ds_temp.metpy.parse_cf()

            init_time = init_time.values
            init_time = dt.utcfromtimestamp(init_time.tolist()/1e9)
            init_time_str = init_time.strftime('%Y-%m-%d %H:%MZ')
            init_time_str_save = init_time.strftime('%Y-%m-%d-%H')
            valid_time = valid_time.values
            valid_time = dt.utcfromtimestamp(valid_time.tolist()/1e9)
            valid_time_str = valid_time.strftime('%Y-%m-%d-%H:%MZ')
            fc_hr = valid_time-init_time
            fc_hr = fc_hr/3600
            fc_hr = fc_hr.seconds
            imgdir = f'images/{var}/{plot_region}/{init_time_str_save}/'
            if not os.path.exists(imgdir):
                os.makedirs(imgdir)
            
            
            dpt = ds_dpt['d2m']
            dpt = (dpt.squeeze()-273.15)*9/5+32
        
            lon,lat = dpt.metpy.coordinates('x', 'y')

            #For filled contour
            #CT = ax1.pcolormesh(lon,lat,t2m, cmap=cmap_temp, norm=norm_temp, shading='goaround', zorder=2, transform=proj)
            CT = ax1.contourf(lon,lat,dpt, cmap=cmap_temp, norm=norm_temp, zorder=2, levels=bounds_temp, transform=proj)


            #Can keep these lines uncommented out so we get a filled colorbar no matter if we use contour or contourf
            sm = plt.cm.ScalarMappable(norm=norm_temp, cmap = CT.cmap)
            sm.set_array([])

            cbar = fig.colorbar(sm, orientation = 'horizontal', aspect = 80, ax = ax1, pad = 0.02,
                    extendrect=False, ticks = range(-50,120,10), fraction=0.046, shrink=0.95)
            cbar.set_label('Dewpoint ($^\circ$F)', fontsize=12)

            ax1.set_title('\n GFS Init: '+init_time_str,fontsize=8,loc='right')

            plt.title('GFS 2-meter Dewpoint Temperature [deg F]\n', fontsize=11, fontweight='bold',loc='left')
            #plt.title('GFS 500mb Geopotential Height (dam), Absolute Vorticity ($10^{-5}s^{-1}$), and Wind (kt)\n', fontsize=11, fontweight='bold',loc='left')
            plt.suptitle('Valid: '+valid_time_str+', Forecast Hour: [' +str(fc_hr)+ ']\n', x=0.232, y=0.883, fontsize=8, zorder=999)
        
            print('Done Plotting')
            #gl = ax1.gridlines(crs =ccrs.PlateCarree(), draw_labels=True,
            #          linewidth=2, color='gray', alpha=0)

            #gl.top_labels = False
            #gl.right_labels = False

#################### TOTAL PRECIPIATION ############################            
        if var == 'acp':
            cmap_acp, norm_acp, acp_levs = cmap_atmos.precip_nws()
            ds_acp =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'stepType': 'accum', 'typeOfLevel': 'surface'})
            
            valid_time = ds_acp['valid_time']
            init_time = ds_acp['time']
            #data1 = ds_temp.metpy.parse_cf()

            init_time = init_time.values
            init_time = dt.utcfromtimestamp(init_time.tolist()/1e9)
            init_time_str = init_time.strftime('%Y-%m-%d %H:%MZ')
            init_time_str_save = init_time.strftime('%Y-%m-%d-%H')
            valid_time = valid_time.values
            valid_time = dt.utcfromtimestamp(valid_time.tolist()/1e9)
            valid_time_str = valid_time.strftime('%Y-%m-%d-%H:%MZ')
            fc_hr = valid_time-init_time
            fc_hr = fc_hr/3600
            fc_hr = fc_hr.seconds
            imgdir = f'images/{var}/{plot_region}/{init_time_str_save}/'
            if not os.path.exists(imgdir):
                os.makedirs(imgdir)
            
            
            acp = ds_acp['tp']
            acp = acp.squeeze()*0.04
        
            lon,lat = acp.metpy.coordinates('x', 'y')

            
            #For filled contour
            #CT = ax1.pcolormesh(lon,lat,t2m, cmap=cmap_temp, norm=norm_temp, shading='goaround', zorder=2, transform=proj)
            CT = ax1.contourf(lon,lat,acp, cmap=cmap_acp, norm=norm_acp, zorder=2, levels=acp_levs, transform=proj)


            #Can keep these lines uncommented out so we get a filled colorbar no matter if we use contour or contourf
            sm = plt.cm.ScalarMappable(norm=norm_acp, cmap = CT.cmap)
            sm.set_array([])

            cbar = fig.colorbar(sm, orientation = 'horizontal', aspect = 80, ax = ax1, pad = 0.02,
                    extendrect=False, fraction=0.046, shrink=0.95)
            cbar.set_label('Precipiation (in)', fontsize=12)

            ax1.set_title('\n GFS Init: '+init_time_str,fontsize=8,loc='right')

            plt.title('GFS Total Accumulated Precipiation [in]\n', fontsize=11, fontweight='bold',loc='left')
            #plt.title('GFS 500mb Geopotential Height (dam), Absolute Vorticity ($10^{-5}s^{-1}$), and Wind (kt)\n', fontsize=11, fontweight='bold',loc='left')
            plt.suptitle('Valid: '+valid_time_str+', Forecast Hour: [' +str(fc_hr)+ ']\n', x=0.232, y=0.883, fontsize=8, zorder=999)
        
            print('Done Plotting')
            #gl = ax1.gridlines(crs =ccrs.PlateCarree(), draw_labels=True,
            #          linewidth=2, color='gray', alpha=0)

            #gl.top_labels = False
            #gl.right_labels = False
##################################### CATEGORICAL PRECIPIATION ##################################
            
        if var == 'cat_precip':
            ds_cat =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'surface'})
            
            valid_time = ds_cat['valid_time']
            init_time = ds_cat['time']
            #data1 = ds_temp.metpy.parse_cf()

            init_time = init_time.values
            init_time = dt.utcfromtimestamp(init_time.tolist()/1e9)
            init_time_str = init_time.strftime('%Y-%m-%d %H:%MZ')
            init_time_str_save = init_time.strftime('%Y-%m-%d-%H')
            valid_time = valid_time.values
            valid_time = dt.utcfromtimestamp(valid_time.tolist()/1e9)
            valid_time_str = valid_time.strftime('%Y-%m-%d-%H:%MZ')
            fc_hr = valid_time-init_time
            fc_hr = fc_hr/3600
            fc_hr = fc_hr.seconds
            imgdir = f'images/{var}/{plot_region}/{init_time_str_save}/'
            if not os.path.exists(imgdir):
                os.makedirs(imgdir)
            
            
            
            crain = ds_cat['crain'].squeeze()
            cfzra = ds_cat['cfrzr'].squeeze()
            cicep = ds_cat['cicep'].squeeze()
            csnow = ds_cat['csnow'].squeeze()
            
            lon,lat = crain.metpy.coordinates('x', 'y')

            
            ip_zr = np.array(cicep + cfzra)
            ip_zr = np.where(ip_zr == 1, 0, ip_zr)
            ip_zr = np.where(ip_zr == 2, 1, ip_zr)

            ip_sn = np.array(cicep + csnow)
            ip_sn = np.where(ip_sn == 1, 0, ip_sn)
            ip_sn = np.where(ip_sn == 2, 1, ip_sn)

            ra_sn = np.array(crain + csnow)
            ra_sn = np.where(ra_sn == 1, 0, ra_sn)
            ra_sn = np.where(ra_sn == 2, 1, ra_sn)

            ra_ip = np.array(crain + cicep)
            ra_ip = np.where(ra_ip == 1, 0, ra_ip)
            ra_ip = np.where(ra_ip == 2, 1, ra_ip)

            zr_sn = np.array(cfzra + csnow)
            zr_sn = np.where(zr_sn == 1, 0, zr_sn)
            zr_sn = np.where(zr_sn == 2, 1, zr_sn)

            zr_ip_sn = np.array(cfzra + csnow + cicep)
            zr_ip_sn = np.where(zr_ip_sn == 1, 0, zr_ip_sn)
            zr_ip_sn = np.where(zr_ip_sn == 2, 0, zr_ip_sn)
            zr_ip_sn = np.where(zr_ip_sn == 3, 1, zr_ip_sn)

            ra_ip_sn = np.array(crain + csnow + cicep)
            ra_ip_sn = np.where(ra_ip_sn == 1, 0, ra_ip_sn)
            ra_ip_sn = np.where(ra_ip_sn == 2, 0, ra_ip_sn)
            ra_ip_sn = np.where(ra_ip_sn == 3, 1, ra_ip_sn)
            
                        #For filled contour
            ax1.contourf(lon, lat, crain, cmap = cmap_rain, transform = ccrs.PlateCarree(), zorder = 2)
            ax1.contourf(lon, lat, csnow, cmap = cmap_snow, transform = ccrs.PlateCarree(), zorder = 2)
            ax1.contourf(lon, lat, cicep, cmap = cmap_icep, transform = ccrs.PlateCarree(), zorder = 2)
            ax1.contourf(lon, lat, cfzra, cmap = cmap_fzra, transform = ccrs.PlateCarree(), zorder = 2)
            ax1.contourf(lon, lat, ip_zr, cmap = cmap_zr_ip, transform = ccrs.PlateCarree(), zorder = 2)
            ax1.contourf(lon, lat, ip_sn, cmap = cmap_sn_ip, transform = ccrs.PlateCarree(), zorder = 2)
            ax1.contourf(lon, lat, ra_sn, cmap = cmap_sn_ra, transform = ccrs.PlateCarree(), zorder = 2)
            ax1.contourf(lon, lat, ra_ip, cmap = cmap_ip_ra, transform = ccrs.PlateCarree(), zorder = 2)
            ax1.contourf(lon, lat, zr_sn, cmap = cmap_zr_sn, transform = ccrs.PlateCarree(), zorder = 2)
            ax1.contourf(lon, lat, zr_ip_sn, cmap = cmap_zr_ip_sn, transform = ccrs.PlateCarree(), zorder = 2)
            ax1.contourf(lon, lat, ra_ip_sn, cmap = cmap_ra_ip_sn, transform = ccrs.PlateCarree(), zorder = 2)


            #Can keep these lines uncommented out so we get a filled colorbar no matter if we use contour or contourf
            #sm = plt.cm.ScalarMappable(cmap = CT.cmap)
            #sm.set_array([])
            
                            #Colorbar
            cbar = plt.colorbar(cbar_ptype, orientation = 'horizontal', ticks = ptype_ticks, pad = 0.02, aspect = 80, ax=ax1, fraction=0.046, extendrect=False, shrink=0.95)
            cbar.ax.set_xticklabels(ptype_labels)
            cbar.ax.set_xlabel("Precipitation Type",fontsize=12)



            ax1.set_title('\n GFS Init: '+init_time_str,fontsize=8,loc='right')

            plt.title('GFS Precipiation Type\n', fontsize=11, fontweight='bold',loc='left')
            #plt.title('GFS 500mb Geopotential Height (dam), Absolute Vorticity ($10^{-5}s^{-1}$), and Wind (kt)\n', fontsize=11, fontweight='bold',loc='left')
            plt.suptitle('Valid: '+valid_time_str+', Forecast Hour: [' +str(fc_hr)+ ']\n', x=0.232, y=0.883, fontsize=8, zorder=999)
        
            print('Done Plotting')
            #gl = ax1.gridlines(crs =ccrs.PlateCarree(), draw_labels=True,
            #          linewidth=2, color='gray', alpha=0)

            #gl.top_labels = False
            #gl.right_labels = False

################## TEMPERATURES ##############################
        if '250_wind' in var:

            
            #opening the data
            
            lev=250.0
            ds_avort =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'isobaricInhPa'}).sel(isobaricInhPa = lev)
            u = ds_avort['u'].squeeze()*1.94384449
            v = ds_avort['v'].squeeze()*1.94384449
            title = '250mb'

                
                
            valid_time = ds_avort['valid_time']
            init_time = ds_avort['time']
            #data1 = ds_temp.metpy.parse_cf()

            init_time = init_time.values
            init_time = dt.utcfromtimestamp(init_time.tolist()/1e9)
            init_time_str = init_time.strftime('%Y-%m-%d %H:%MZ')
            init_time_str_save = init_time.strftime('%Y-%m-%d-%H')
            valid_time = valid_time.values
            valid_time = dt.utcfromtimestamp(valid_time.tolist()/1e9)
            valid_time_str = valid_time.strftime('%Y-%m-%d-%H:%MZ')
            fc_hr = valid_time-init_time
            fc_hr = fc_hr/3600
            fc_hr = fc_hr.seconds
            imgdir = f'images/{var}/{plot_region}/{init_time_str_save}/'
            if not os.path.exists(imgdir):
                os.makedirs(imgdir)
            
            lon,lat = u.metpy.coordinates('x', 'y')
            u = u*units.knots
            v = v*units.knots

            wind = mpcalc.wind_speed(u , v)
            cmap = 'afmhot_r'
            norm = mpl.colors.Normalize(10,150,10)
            levs = np.arange(10,150,10)
            #For filled contour
            #CT = ax1.pcolormesh(lon,lat,t2m, cmap=cmap_temp, norm=norm_temp, shading='goaround', zorder=2, transform=proj)
            CT = ax1.contourf(lon,lat,wind, cmap=cmap, norm=norm, zorder=2, levels=levs, transform=proj)

            skip = int(0.8*plot_regions_dict[plot_region]['windbarb_skip'])
            wind_slice = slice(None, None, skip)
            
            lon = x.values
            lat = y.values
            u = u.values
            v = v.values

            ax1.barbs(lon[wind_slice],lat[wind_slice],u[wind_slice,wind_slice],v[wind_slice,wind_slice], length=5, linewidth=0.35, color='dimgrey', zorder=998, transform=proj)

            #For unfilled contour
            #sigma = 0.0 # this depends on how noisy your data is, adjust as necessary
            #Tsmth = mpcalc.smooth_gaussian(temp, sigma)
            #CT = ax1.contour(lon,lat,Tsmth,levels=np.arange(-60,130+10,10), transform = ccrs.PlateCarree(), cmap=cmap, norm=norm)
            #ax1.clabel(CT, inline_spacing=0.2)

            #Can keep these lines uncommented out so we get a filled colorbar no matter if we use contour or contourf
            sm = plt.cm.ScalarMappable(norm=norm, cmap = CT.cmap)
            sm.set_array([])

            cbar = fig.colorbar(sm, orientation = 'horizontal', aspect = 80, ax = ax1, pad = 0.02,
                    extendrect=False, ticks = range(-60,131,10), fraction=0.046, shrink=0.95)
            cbar.set_label('Wind Speed (kt)', fontsize=12)

            ax1.set_title('\n GFS Init: '+init_time_str,fontsize=8,loc='right')

            plt.title(f'GFS {title} Wind [kt, fill], Wind Barbs [kt]\n', fontsize=11, fontweight='bold',loc='left')
            #plt.title('GFS 500mb Geopotential Height (dam), Absolute Vorticity ($10^{-5}s^{-1}$), and Wind (kt)\n', fontsize=11, fontweight='bold',loc='left')
            plt.suptitle('Valid: '+valid_time_str+', Forecast Hour: [' +str(fc_hr)+ ']\n', x=0.232, y=0.883, fontsize=8, zorder=999)
        
            print('Done Plotting')
            
#################### PRECIPIATION RATE ############################            
        if var == 'precip_rate':
            #cmap_acp, norm_acp, acp_levs = cmap_atmos.precip_nws()
            ptype_series = []
            #downloading the data
            if x <= 120:
                b = 5
                g = 1
                r = 6
            if x > 120:
                b = 3
                g = 3
                r = 2
            for y in range(x-b,x+1,g):
                if len(str(y)) == 1:
                    y2 = f'00{y}'
                if len(str(y)) == 2:
                    y2 = f'0{y}'
                if len(str(y)) == 3:
                    y2 = y
                    
                ds = wget.download(f'https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?file=gfs.t{time}z.pgrb2.0p25.f{y2}&lev_500_mb=on&lev_1000_mb=on&lev_mean_sea_level=on&lev_surface=on&var_CFRZR=on&var_CICEP=on&var_CRAIN=on&var_CSNOW=on&var_HGT=on&var_MSLET=on&var_PRATE=on&subregion=&leftlon=229&rightlon=297.5&toplat=55.5&bottomlat=20&dir=%2Fgfs.{date}%2F{time}%2Fatmos')
                #print('Downloaded Url '+str(new_url[y]))
                ptype_series.append(ds)
            precip = []
            for l in range(0,r):
                ds_pr =  xr.open_mfdataset(ptype_series[l], engine='cfgrib',filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'surface'})
                precip.append(ds_pr)
            
            #MSLP
            ds_h =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'meanSea'})
            mslp = ds_h['mslet'].squeeze()*.01
            
            #500mb Height
            ds_h5 =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'isobaricInhPa'}).sel(isobaricInhPa = 500)
            ds_h1 =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'isobaricInhPa'}).sel(isobaricInhPa = 1000)
            h5 = ds_h5['gh'].squeeze()*.1
            h1 = ds_h1['gh'].squeeze()*.1
            
            thick = h5-h1
            
            precip_con = xr.concat(precip, dim='time')
            precip_mean = precip_con.mean(dim = 'time')
            
            valid_time = ds_pr['valid_time']
            init_time = ds_pr['time']
            #data1 = ds_temp.metpy.parse_cf()

            init_time = init_time.values
            init_time = dt.utcfromtimestamp(init_time.tolist()/1e9)
            init_time_str = init_time.strftime('%Y-%m-%d %H:%MZ')
            init_time_str_save = init_time.strftime('%Y-%m-%d-%H')
            valid_time = valid_time.values
            valid_time = dt.utcfromtimestamp(valid_time.tolist()/1e9)
            valid_time_str = valid_time.strftime('%Y-%m-%d-%H:%MZ')
            fc_hr = valid_time-init_time
            fc_hr = fc_hr/3600
            fc_hr = fc_hr.seconds
            imgdir = f'images/{var}/{plot_region}/{init_time_str_save}/'
            if not os.path.exists(imgdir):
                os.makedirs(imgdir)
            
            
            prate = precip_mean['prate']
            prate = (prate.squeeze()*86400*.0393701)/24

        
            lon,lat = prate.metpy.coordinates('x', 'y')

            
            #For filled contour
            CT = ax1.contourf(lon,lat,prate, cmap=cmap_prate, norm=norm_prate, zorder=2, transform=proj, levels=bounds_prate)
            
            #MSLP
            levels = np.arange(0,1080,2)
            hc = ax1.contour(lon,lat,mslp,colors='k', levels = levels,linewidths=0.75, zorder=995, transform=proj)
            ax1.clabel(hc, inline_spacing=0.2, fontsize='small', zorder=999)
            
            #500mb height
            levels = np.arange(480,620,6)
            hwarm = thick.where(thick.values > 540)
            hcold = thick.where(thick.values <= 540)
            hcold = np.nan_to_num(hcold, nan = 0)
            h5warm = ax1.contour(lon,lat,hwarm,colors='red', levels = levels,linewidths=0.75, zorder=995, transform=proj, linestyles='dashed')
            ax1.clabel(h5warm, inline_spacing=0.2, fontsize='small', zorder=999)
            h5cold = ax1.contour(lon,lat,hcold,colors='blue', levels = levels,linewidths=0.75, zorder=995, transform=proj, linestyles='dashed')
            ax1.clabel(h5cold, inline_spacing=0.2, fontsize='small', zorder=999)


            cbar = fig.colorbar(CT, orientation = 'horizontal', aspect = 80, ax = ax1, pad = 0.02,
                    extendrect=False, fraction=0.046, norm=norm_prate, ticks=[0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.75,1], shrink=0.95)
            cbar.set_label('Precipiation (in)', fontsize=12)

            ax1.set_title('\n GFS Init: '+init_time_str,fontsize=8,loc='right')

            plt.title('GFS 6-Hour Average Precipitation Rate [in/hr], MSLP [mb], 1000-500mb Thickness (dam)\n', fontsize=11, fontweight='bold',loc='left')
            #plt.title('GFS 500mb Geopotential Height (dam), Absolute Vorticity ($10^{-5}s^{-1}$), and Wind (kt)\n', fontsize=11, fontweight='bold',loc='left')
            plt.suptitle('Valid: '+valid_time_str+', Forecast Hour: [' +str(fc_hr)+ ']\n', x=0.232, y=0.883, fontsize=8, zorder=999)
        
            print('Done Plotting')
            #gl = ax1.gridlines(crs =ccrs.PlateCarree(), draw_labels=True,
            #          linewidth=2, color='gray', alpha=0)

            #gl.top_labels = False
            #gl.right_labels = False

#################### PRECIPIATION TYPE RATE ############################            
        if var == 'ptype_rate':
            #cmap_acp, norm_acp, acp_levs = cmap_atmos.precip_nws()
            ptype_series = []
            #downloading the data

            if x <= 240:
                b = 5
                g = 1
                r = 6
            if x > 240:
                b = 3
                g = 3
                r = 2
            for y in range(x-b,x+1,g):
                if len(str(y)) == 1:
                    y2 = f'00{y}'
                if len(str(y)) == 2:
                    y2 = f'0{y}'
                if len(str(y)) == 3:
                    y2 = y
                #ds = wget.download(f'https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?file=gfs.t{time}z.pgrb2.0p25.f{y2}&lev_500_mb=on&lev_1000_mb=on&lev_mean_sea_level=on&lev_surface=on&var_CFRZR=on&var_CICEP=on&var_CRAIN=on&var_CSNOW=on&var_HGT=on&var_MSLET=on&var_PRATE=on&subregion=&leftlon=229&rightlon=297.5&toplat=55.5&bottomlat=20&dir=%2Fgfs.{date}%2F{time}%2Fatmos')
                ds = wget.download(f'https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?file=gfs.t12z.pgrb2.0p25.f{y2}&lev_500_mb=on&lev_1000_mb=on&lev_mean_sea_level=on&lev_surface=on&var_CFRZR=on&var_CICEP=on&var_CRAIN=on&var_CSNOW=on&var_HGT=on&var_MSLET=on&var_PRATE=on&subregion=&leftlon=229&rightlon=297.5&toplat=55.5&bottomlat=20&dir=%2Fgfs.{date}%2F12%2Fatmos')

                ptype_series.append(ds)
            precip = []
            for l in range(0,r):
                ds_pr =  xr.open_mfdataset(ptype_series[l], engine='cfgrib',filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'surface'})
                precip.append(ds_pr)

            valid_time = ds_pr['valid_time']
            init_time = ds_pr['time']
            #data1 = ds_temp.metpy.parse_cf()

            init_time = init_time.values
            init_time = dt.utcfromtimestamp(init_time.tolist()/1e9)
            init_time_str = init_time.strftime('%Y-%m-%d %H:%MZ')
            init_time_str_save = init_time.strftime('%Y-%m-%d-%H')
            valid_time = valid_time.values
            valid_time = dt.utcfromtimestamp(valid_time.tolist()/1e9)
            valid_time_str = valid_time.strftime('%Y-%m-%d-%H:%MZ')
            fc_hr = valid_time-init_time
            fc_hr = fc_hr/3600
            fc_hr = fc_hr.seconds
            imgdir = f'images/{var}/{plot_region}/{init_time_str_save}/'
            if not os.path.exists(imgdir):
                os.makedirs(imgdir)
                
                
            #MSLP
            ds_h =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'meanSea'})
            mslp = ds_h['mslet'].squeeze()*.01
            
            #Thickness
            ds_h5 =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'isobaricInhPa'}).sel(isobaricInhPa = 500)
            ds_h1 =  xr.open_mfdataset(ds, engine='cfgrib',filter_by_keys={'typeOfLevel': 'isobaricInhPa'}).sel(isobaricInhPa = 1000)
            h5 = ds_h5['gh'].squeeze()*.1
            h1 = ds_h1['gh'].squeeze()*.1
            
            thick = h5-h1
            
            precip_con = xr.concat(precip, dim='time')
            precip_mean = precip_con.mean(dim = 'time')
            
            crain = precip_mean['crain'].values
            csnow = precip_mean['csnow'].values
            cfrzr = precip_mean['cfrzr'].values
            cicep = precip_mean['cicep'].values
            
            crain = np.where((crain > csnow) &  (crain>cfrzr) & (crain>cicep),1,0)
            csnow = np.where((csnow > crain) &  (csnow>cfrzr) & (csnow>cicep),1,0)
            cicep = np.where((cicep > csnow) &  (cicep>cfrzr) & (cicep>crain),1,0)
            cfrzr = np.where((cfrzr > csnow) &  (cfrzr>crain) & (cfrzr>cicep),1,0)
            
            #finding values of last frame to determine ptype if there is a time in the average of the 6 hours
            last_frame = precip[5]
            rain_last = last_frame['crain']
            snow_last = last_frame['csnow']
            icep_last = last_frame['cicep']
            frzr_last = last_frame['cfrzr']

            rain_mix =np.where(((crain == csnow) | (crain == cfrzr) | (crain == cicep)) & (crain == 1) & (rain_last == 1),1,0)
            frzr_mix =np.where(((cfrzr == csnow) | (cfrzr == crain) | (cfrzr == cicep)) & (cfrzr == 1) & (frzr_last == 1),1,0)
            snow_mix =np.where(((csnow == crain) | (csnow == cfrzr) | (csnow == cicep)) & (csnow == 1) & (snow_last == 1),1,0)
            icep_mix =np.where(((cicep == csnow) | (cicep == cfrzr) | (cicep == crain)) & (cicep == 1) & (icep_last == 1),1,0)
            
            rain = crain+rain_mix
            rain = np.where((rain>0),1,0)
            snow = csnow+snow_mix
            snow = np.where((snow>0),1,0)
            icep = cicep+icep_mix
            icep = np.where((icep>0),1,0)
            frzr = cfrzr+frzr_mix
            frzr = np.where((frzr>0),1,0)
            
            #finding precip rate
            prate = precip_mean['prate']
            prate = (prate.squeeze()*86400*.0393701)/24
            
            rain = np.where((rain==1),prate,0)
            snow = np.where((snow==1),prate,0)
            icep = np.where((icep==1),prate,0)
            frzr = np.where((frzr==1),prate,0)

        
            lon,lat = prate.metpy.coordinates('x', 'y')

            
            #For filled contour
            rain_c = ax1.contourf(lon,lat,rain, cmap=cmap_prate, norm=norm_prate, zorder=2, transform=proj, levels=bounds_prate)
            snow_c = ax1.contourf(lon,lat,snow, cmap=cmap_snow_c, norm=norm_w, zorder=2, transform=proj, levels=bounds_w)
            icep_c = ax1.contourf(lon,lat,icep, cmap=cmap_icep_c, norm=norm_w, zorder=2, transform=proj, levels=bounds_w)
            frzr_c = ax1.contourf(lon,lat,frzr, cmap=cmap_frzr_c, norm=norm_w, zorder=2, transform=proj, levels=bounds_w)
            
            #MSLP
            levels = np.arange(0,1080,2)
            hc = ax1.contour(lon,lat,mslp,colors='k', levels = levels,linewidths=0.75, zorder=995, transform=proj)
            ax1.clabel(hc, inline_spacing=0.2, fontsize='small', zorder=999)
            
            #500mb height
            levels_warm = np.arange(546,620,6)
            levels_cold = np.arange(480,540,6)
            hwarm = thick.where(thick.values > 540)
            hcold = thick.where(thick.values <= 540)
            hcold = np.nan_to_num(hcold, nan = 0)
            h5warm = ax1.contour(lon,lat,hwarm,colors='red', levels = levels_warm,linewidths=0.75, zorder=995, transform=proj, linestyles='dashed')
            ax1.clabel(h5warm, inline_spacing=0.2, fontsize='small', zorder=999)
            h5cold = ax1.contour(lon,lat,hcold,colors='blue', levels = levels_cold,linewidths=0.75, zorder=995, transform=proj, linestyles='dashed')
            ax1.clabel(h5cold, inline_spacing=0.2, fontsize='small', zorder=999)
            
            ax1.set_title('\n GFS Init: '+init_time_str,fontsize=8,loc='right')

            plt.title('GFS 6-Hour Average Precipitation Rate [in/hr], MSLP [mb], 1000-500mb Thickness (dam)\n', fontsize=11, fontweight='bold',loc='left')
            #plt.title('GFS 500mb Geopotential Height (dam), Absolute Vorticity ($10^{-5}s^{-1}$), and Wind (kt)\n', fontsize=11, fontweight='bold',loc='left')
            plt.suptitle('Valid: '+valid_time_str+', Forecast Hour: [' +str(fc_hr)+ ']\n', x=0.232, y=0.867, fontsize=8, zorder=999)

            
            cax_rain = fig.add_axes([0.16, 0.12, .15, 0.02])
            cbar_rain = fig.colorbar(rain_c, orientation = 'horizontal', aspect = 80, cax=cax_rain, pad = 0.02,
                    extendrect=False, fraction=0.046, norm=norm_prate, ticks=[0.05,0.1,0.25,0.5,1])
            cbar_rain.set_label('Rain', fontsize=12, x=-0.12, labelpad = -27)


            cax_snow = fig.add_axes([0.36, 0.12, .15, 0.02])
            cbar_snow = fig.colorbar(snow_c, orientation = 'horizontal', aspect = 80, cax=cax_snow, pad = 0.02,
                    extendrect=False, fraction=0.046, norm=norm_w, ticks=[0.05,0.1,0.25,0.5])
            cbar_snow.set_label('Snow', fontsize=12, x=-0.14, labelpad = -27)

            cax_icep = fig.add_axes([0.55, 0.12, .15, 0.02])
            cbar_icep = fig.colorbar(icep_c, orientation = 'horizontal', aspect = 80, cax=cax_icep, pad = 0.02,
                    extendrect=False, fraction=0.046, norm=norm_w, ticks=[0.05,0.1,0.25,0.5])
            cbar_icep.set_label('Sleet', fontsize=12, x=-0.12, labelpad = -27)

            cax_frzr = fig.add_axes([0.74, 0.12, .15, 0.02])
            cbar_frzr = fig.colorbar(frzr_c, orientation = 'horizontal', aspect = 80, cax=cax_frzr, pad = 0.02,
                    extendrect=False, fraction=0.046, norm=norm_w, ticks=[0.05,0.1,0.25,0.5])
            cbar_frzr.set_label('Frzr', fontsize=12, x=-0.12, labelpad = -27)

        
            print('Done Plotting')
            #gl = ax1.gridlines(crs =ccrs.PlateCarree(), draw_labels=True,
            #          linewidth=2, color='gray', alpha=0)

            #gl.top_labels = False
            #gl.right_labels = False
            
#################### MAX GUST ############################            
        if var == 'max_gust':
            #cmap_acp, norm_acp, acp_levs = cmap_atmos.precip_nws()
            gust_series = []
            #downloading the data

            if x <= 240:
                b = 5
                g = 1
                r = 6
            if x > 240:
                b = 3
                g = 3
                r = 2
            for y in range(x-b,x+1,g):
                if len(str(y)) == 1:
                    y2 = f'00{y}'
                if len(str(y)) == 2:
                    y2 = f'0{y}'
                if len(str(y)) == 3:
                    y2 = y
                #ds = wget.download(f'https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?file=gfs.t{time}z.pgrb2.0p25.f{y2}&lev_500_mb=on&lev_1000_mb=on&lev_mean_sea_level=on&lev_surface=on&var_CFRZR=on&var_CICEP=on&var_CRAIN=on&var_CSNOW=on&var_HGT=on&var_MSLET=on&var_PRATE=on&subregion=&leftlon=229&rightlon=297.5&toplat=55.5&bottomlat=20&dir=%2Fgfs.{date}%2F{time}%2Fatmos')
                ds = wget.download(f'https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?file=gfs.t{time}z.pgrb2.0p25.f{y2}&lev_surface=on&var_GUST=on&subregion=&leftlon=229&rightlon=297.5&toplat=55.5&bottomlat=20&dir=%2Fgfs.{date}%2F{time}%2Fatmos')

                gust_series.append(ds)
            gust = []
            for l in range(0,r):
                ds_gust =  xr.open_mfdataset(gust_series[l], engine='cfgrib',filter_by_keys={'stepType': 'instant', 'typeOfLevel': 'surface'})
                gust.append(ds_gust)

            valid_time = ds_gust['valid_time']
            init_time = ds_gust['time']
            #data1 = ds_temp.metpy.parse_cf()

            init_time = init_time.values
            init_time = dt.utcfromtimestamp(init_time.tolist()/1e9)
            init_time_str = init_time.strftime('%Y-%m-%d %H:%MZ')
            init_time_str_save = init_time.strftime('%Y-%m-%d-%H')
            valid_time = valid_time.values
            valid_time = dt.utcfromtimestamp(valid_time.tolist()/1e9)
            valid_time_str = valid_time.strftime('%Y-%m-%d-%H:%MZ')
            fc_hr = valid_time-init_time
            fc_hr = fc_hr/3600
            fc_hr = fc_hr.seconds
            imgdir = f'images/{var}/{plot_region}/{init_time_str_save}/'
            if not os.path.exists(imgdir):
                os.makedirs(imgdir)
                
            
            gust_con = xr.concat(gust, dim='time')
            gust_max = gust_con.max(dim = 'time')
            
            gust_max = gust_max['gust'].squeeze()*2.23694

            
            lon,lat = gust_max.metpy.coordinates('x', 'y')

            
            gust_f = ax1.contourf(lon,lat,gust_max, cmap=cmap_gust, norm=norm_gust, zorder=2, transform=proj, levels=bounds_gust)
            
            ax1.set_title('\n GFS Init: '+init_time_str,fontsize=8,loc='right')

            plt.title('GFS 6-Hour 10m Max Wind Gust [mph]\n', fontsize=11, fontweight='bold',loc='left')
            #plt.title('GFS 500mb Geopotential Height (dam), Absolute Vorticity ($10^{-5}s^{-1}$), and Wind (kt)\n', fontsize=11, fontweight='bold',loc='left')
            plt.suptitle('Valid: '+valid_time_str+', Forecast Hour: [' +str(fc_hr)+ ']\n', x=0.232, y=0.883, fontsize=8, zorder=999)
            
            cbar = fig.colorbar(gust_f, orientation = 'horizontal', aspect = 80, ax = ax1, pad = 0.02,
                    extendrect=False, ticks = [10,20,30,39,50,60,74,95,110,129,155], fraction=0.046, shrink=0.95)
            cbar.set_label('Gusts (mph)', fontsize=12)
            
            print('Done Plotting')
            
        plt.savefig(imgdir+f'gfs_6hr_{var}_'+valid_time_str+'_hour_'+str(fc_hr)+'.png', bbox_inches = 'tight')
        print('Saved Figure')
        #plt.clf()
        
    #delete all fikes
    files = [file for file in glob.glob("/nfs/home11/ugrad/2020/mb127935/MetPy/Xarray/Model_Data/GFS/model_variables/*.pgrb2*")]
    for file in files:
        os.remove(file)
#end = t.time()
#print(end - start)

Ignoring index file '/nfs/spare11/mb127935/MetPy/Xarray/Model_Data/GFS/model_variables/gfs.t12z.pgrb2.0p25 (5).f096.923a8.idx' incompatible with GRIB file
Ignoring index file '/nfs/spare11/mb127935/MetPy/Xarray/Model_Data/GFS/model_variables/gfs.t12z.pgrb2.0p25 (5).f096.923a8.idx' incompatible with GRIB file
Ignoring index file '/nfs/spare11/mb127935/MetPy/Xarray/Model_Data/GFS/model_variables/gfs.t12z.pgrb2.0p25 (5).f096.923a8.idx' incompatible with GRIB file
/knight/anaconda_jan21/envs/oct21/lib/python3.9/site-packages/cartopy/mpl/geoaxes.py:1666: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


Done Plotting
Saved Figure


Ignoring index file '/nfs/spare11/mb127935/MetPy/Xarray/Model_Data/GFS/model_variables/gfs.t12z.pgrb2.0p25 (11).f096.923a8.idx' incompatible with GRIB file
Ignoring index file '/nfs/spare11/mb127935/MetPy/Xarray/Model_Data/GFS/model_variables/gfs.t12z.pgrb2.0p25 (11).f096.923a8.idx' incompatible with GRIB file
Ignoring index file '/nfs/spare11/mb127935/MetPy/Xarray/Model_Data/GFS/model_variables/gfs.t12z.pgrb2.0p25 (11).f096.923a8.idx' incompatible with GRIB file
/knight/anaconda_jan21/envs/oct21/lib/python3.9/site-packages/cartopy/mpl/geoaxes.py:1666: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


Done Plotting
Saved Figure
Done Plotting
Saved Figure
